### Analysis Loop

Loop over the final sample with the dbscan parameters and deliver the OC stars for analysis.

# Test with A_G

In [11]:
using PyCall
using Distributions
using Statistics
using DataFrames

using Distances
using Random , Printf

rootdir = ENV["GAIA_ROOT"]

push!(LOAD_PATH,"$rootdir/run/src")
using GaiaClustering

import PyPlot , CSV

wdir    = "$rootdir/products"
votdir  = "$rootdir/products/votable"
ocdir   = "$rootdir/products/oc2"

cd(wdir)

In [12]:
function getdata(filevot, wghtblck)
    voname = filevot

    data       = read_votable(voname)
    df         = filter_data(data)
    dfcart     = add_cartesian(df)
    blck       = [[1,2,3],[4,5], [6,7,8]]
    #wghtblck   = [5,4,1]
    norm       = "identity"

    dfcartnorm , scale8 = normalization_PerBlock(dfcart, blck, wghtblck , norm, false)
    return(df, dfcart , dfcartnorm)
end

getdata (generic function with 1 method)

In [13]:
## Check if OC was extracted
function _check_extraction(votname, fileres)
    try
        res = CSV.read(fileres, delim=";") 
        if votname in res[:votname]
            return(true)
        else
            return(false)
        end        
    catch
        println("### $fileres will be created...")
        return(false)
    end
end

function _extraction_updt(fileres, votname)
    try
        res = CSV.read(fileres, delim=";")
        newrow = DataFrame(votname=votname)
        append!(res,newrow)
        CSV.write(fileres,res,delim=';')
    catch
        println("## No $fileres file, it will be created...")
        res= DataFrame(votname=votname)
        CSV.write(fileres,res,delim=';')
    end
end
    
## Create the DataFrame to save the cluster...
##
function _export_df(votname, ocdir, df , dfcart, labels , labelmax)
    ra= df.raw[1, labels[labelmax]]
    dec= df.raw[2,labels[labelmax]]
    l= df.data[1, labels[labelmax]]
    b= df.data[2,labels[labelmax]]
    d= df.data[3,labels[labelmax]]
    pmra= df.raw[6, labels[labelmax]]
    pmdec= df.raw[7, labels[labelmax]]
    X= dfcart.data[1, labels[labelmax]]
    Y= dfcart.data[2, labels[labelmax]]
    Z= dfcart.data[3, labels[labelmax]]
    vl= df.data[4,labels[labelmax]]
    vb= df.data[5,labels[labelmax]]
    vrad= df.raw[13,labels[labelmax]]
    gbar= df.raw[10,labels[labelmax]]
    rp= df.raw[11,labels[labelmax]]
    bp= df.raw[12,labels[labelmax]]
    ag= df.raw[14,labels[labelmax]]
    
    oc= DataFrame(ra=ra,dec=dec,l=l,b=b, distance=d,pmra=pmra, pmdec=pmdec, X=X,Y=Y,Z=Z,vl=vl,vb=vb,vrad=vrad,gbar=gbar,rp=rp,bp=bp, ag=ag)
    
    filename= @sprintf("%s/%s-oc.csv",ocdir, votname[1:end-4])
    CSV.write(filename,oc,delim=';')
    @printf("\n## %s created",filename)
end

_export_df (generic function with 1 method)

In [14]:
## Main loop
##

macro main(paramfile, fileres)
    let
        println("## Starting main loop..")
        spl= CSV.read(paramfile, delim= ";")
        println("## $paramfile read..") 
        println("## Check if csv file is ; separated..")
        s=size(spl)
        
        for i in 1:s[1]
            votname = spl[:votname][i]
            println("## Starting with $votname")
            wght= [spl[:w3dm][i],spl[:wvelm][i] ,spl[:whrdm][i]]
            println(wght)
            
            found= _check_extraction(votname, fileres)
            if !found
                df , dfcart , dfcartnorm = getdata(votdir*"/"*votname, wght)
            
                ## get the cluster and plot it
                println("## Extracting the cluster using DBSCAN with:")
                eps = spl[:epsm_1][i]
                min_nei = trunc(Int,spl[:mneim_1][i] + 0.5)  
                min_cl = trunc(Int,spl[:mclm_1][i] + 0.5)
                println("### ϵ : $eps")
                println("### min_neighbor: $min_nei")
                println("### min_cluster : $min_cl")
                labels = clusters(dfcartnorm.data ,eps  , 20, min_nei, min_cl)
                labelmax , nmax = find_cluster_label(labels)
                println("### Label solution: $labelmax")
                println("### N stars: $nmax")
                
                _export_df(votname, ocdir, df , dfcart, labels , labelmax)
                _extraction_updt(fileres, votname)
            end
        end
    end
end

@main (macro with 1 method)

In [ ]:
@time @main("votlist.finalSample_metric0.01.csv" , "votlist.finalSample_metric0.01.extracted.csv")

## Starting main loop..
## votlist.finalSample_metric0.01.csv read..
## Check if csv file is ; separated..
## Starting with Alessi 1-2.1deg.vot
[3.41427, 7.99762, 2.0506]
### votlist.finalSample_metric0.01.extracted.csv will be created...
## Votable /home/stephane/Science/GAIA/products/votable/Alessi 1-2.1deg.vot read


┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = read_votable(::String) at data.jl:47
└ @ GaiaClustering /home/stephane/Science/GAIA/master/src/data.jl:47
┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = read_votable(::String) at data.jl:47
└ @ GaiaClustering /home/stephane/Science/GAIA/master/src/data.jl:47
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = filter_data(::PyObject, ::Array{Float64,1}, ::Array{Int64,1}, ::Array{Float64,1}, ::Array{Float64,1}) at data.jl:83
└ @ GaiaClustering /home/stephane/Science/GAIA/master/src/data.jl:83
┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getprop

┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = filter_data(::PyObject, ::Array{Float64,1}, ::Array{Int64,1}, ::Array{Float64,1}, ::Array{Float64,1}) at data.jl:113
└ @ GaiaClustering /home/stephane/Science/GAIA/master/src/data.jl:113
┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = filter_data(::PyObject, ::Array{Float64,1}, ::Array{Int64,1}, ::Array{Float64,1}, ::Array{Float64,1}) at data.jl:113
└ @ GaiaClustering /home/stephane/Science/GAIA/master/src/data.jl:113


## Filtering done ...
## Stars selected: 36217
## Cartesian transformation done ...
## Normalization identity done...
### [1pc,1pc,1pc,1km/s,1km/s,1mag,1mag,1mag] equivalent to [0.257719, 0.257719, 0.257719, 0.603683, 0.603683, 0.154786, 0.154786, 0.154786]
##
## Extracting the cluster using DBSCAN with:
### ϵ : 1.9699327808371245
### min_neighbor: 7
### min_cluster : 16
### Label solution: 1
### N stars: 178

## /home/stephane/Science/GAIA/products/oc2/Alessi 1-2.1deg-oc.csv created## No votlist.finalSample_metric0.01.extracted.csv file, it will be created...
## Starting with Alessi 12-2.4deg.vot
[4.48411, 8.63144, 1.95928]
